In [ ]:
import cv2
import mediapipe as mp
import math
import numpy as np
import osascript
import time

In [ ]:
mpHands = mp.solutions.hands
hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils #drawing the landmarks

In [ ]:
font                   = cv2.FONT_HERSHEY_SIMPLEX
bottomLeftCornerOfText = (10,500)
fontScale              = 1
fontColor              = (255,255,255)
lineType               = 2

In [ ]:
def update_volume(val):
    vol = "set volume output volume " + str(val)
    osascript.osascript(vol)
    return vol

def get_distance(img, lmList,finger1 = 0, finger2 = 1, handNo = 0, draw=True, r=15, t=3):
    tipPos = [4, 8, 12, 16, 20] #at this position, we can get the landmark of the tips of the 5 fingers
    lmList = lmList
    x1, y1 = lmList[tipPos[finger1]][1:] #get x and y coordinates of index finger
    x2, y2 = lmList[tipPos[finger2]][1:] #get x and y coordinates of middle finger
    length = math.hypot(x2-x1, y2-y1)
    cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
    if draw:
        cv2.line(img, (x1, y1), (x2, y2), (255, 0, 255), t)
        cv2.circle(img, (x1, y1), r, (255, 0, 255), cv2.FILLED)
        cv2.circle(img, (x2, y2), r, (255, 0, 255), cv2.FILLED)
        cv2.circle(img, (cx, cy), r, (0, 0, 255), cv2.FILLED)
    
    return length,img
    
def findHands(img):
    imgRGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    if(results.multi_hand_landmarks):
        for handLms in results.multi_hand_landmarks:
            mpDraw.draw_landmarks(img,handLms, mpHands.HAND_CONNECTIONS)
    return img
    

def get_position(img, handNo = 0): #get all landmark positions
    lmList = []
    xList = []
    yList = []
    
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    if results.multi_hand_landmarks:
        myHand = results.multi_hand_landmarks[handNo]
        for id, lm in enumerate(myHand.landmark):
            h,w,c = img.shape
            #converting those into dimensions 
            cx,cy = int(lm.x*w), int(lm.y*h)
            xList.append(cx)
            yList.append(cy)
            lmList.append([id, cx,cy])
            
    return lmList
            


In [ ]:
cap = cv2.VideoCapture(0)
# volume.GetMute()
# volume.GetMasterVolumeLevel()


while True:
    success, img = cap.read()
    if not success:
        print("Failed to read frame")
        continue
        
    image = findHands(img)
    lmList = get_position(img)
    if len(lmList)!= 0:
        length, img = get_distance(img, lmList)
        
        vol = np.interp(length, [50, 300], [0, 100])
        volBar = np.interp(length, [50, 300], [400, 150])
        volPer = np.interp(length, [50, 300], [0, 100])
        vol = int(vol)
        x = update_volume(vol)

        cv2.rectangle(img, (50, 150), (85, 400), (255, 0, 0), 3)
        cv2.rectangle(img, (50, int(volBar)), (85, 400), (255, 0, 0), cv2.FILLED)
        cv2.putText(img, f'{int(volPer)} %', (40, 450), cv2.FONT_HERSHEY_COMPLEX,
                    1, (255, 0, 0), 3)

    cv2.imshow("Image", image)
    cv2.waitKey(1)
    
cap.release()
cv2.destroyAllWindows()